## 第一部分：了解 nn.Module的基本操作

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

In [2]:
model = models.resnet18()

### 打印出 model底下所有 parameters 的 name 以及對應的 shape 

In [3]:
for name, param in model.named_parameters():
    print(name,param.shape)

conv1.weight torch.Size([64, 3, 7, 7])
bn1.weight torch.Size([64])
bn1.bias torch.Size([64])
layer1.0.conv1.weight torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight torch.Size([64])
layer1.0.bn1.bias torch.Size([64])
layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight torch.Size([64])
layer1.0.bn2.bias torch.Size([64])
layer1.1.conv1.weight torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight torch.Size([64])
layer1.1.bn1.bias torch.Size([64])
layer1.1.conv2.weight torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight torch.Size([64])
layer1.1.bn2.bias torch.Size([64])
layer2.0.conv1.weight torch.Size([128, 64, 3, 3])
layer2.0.bn1.weight torch.Size([128])
layer2.0.bn1.bias torch.Size([128])
layer2.0.conv2.weight torch.Size([128, 128, 3, 3])
layer2.0.bn2.weight torch.Size([128])
layer2.0.bn2.bias torch.Size([128])
layer2.0.downsample.0.weight torch.Size([128, 64, 1, 1])
layer2.0.downsample.1.weight torch.Size([128])
layer2.0.downsample.1.bias torch.Size([128])
layer2.1.conv1.weight tor

### 為了使 forward propagation 加速 並降低 memory 使用量，請將所有 parameters 的requires_grad 關閉，關閉之後執行  forward propagation

In [4]:
input_ = torch.randn(1, 3, 128, 128)

In [5]:
for param in model.parameters():
    param.requires_grad = False

In [6]:
output = model(input_)
print(output.shape)

torch.Size([1, 1000])


## 第二部分：依照指令，以較簡潔的方式搭建出模型

* input_shape = torch.Size([10, 12])
* 先經過一層 nn.Linear(12, 10)
* 經過10層 nn.Linear(10, 10)
* 最後經過 nn.Linear(10, 3) 輸出
* 每一個 nn.Linear過完後要先經過 nn.BatchNorm1d 才能到下一層，輸出層不用


In [7]:
input_ = torch.randn(10,12)
## 示範
Linear = nn.Linear(12,10)
BN = nn.BatchNorm1d(10)

### 第一種

In [8]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.input = nn.Sequential(
                            nn.Linear(12,10),
                            nn.BatchNorm1d(10))
        self.hidden = nn.ModuleList([nn.Sequential(
                            nn.Linear(10,10),
                            nn.BatchNorm1d(10)) for i in range(10)])
        self.output = nn.Linear(10, 3)

    def forward(self, x):
        x = self.input(x)
        for module in self.hidden:
            x = module(x) 
        x = self.output(x)
        return x

In [9]:
model = Model()
input_ = torch.randn(10,12)
output = model(input_)
output

tensor([[-0.7020, -0.1810,  0.7230],
        [-0.7889,  0.5047, -0.6906],
        [-0.6963,  0.3273, -1.1228],
        [ 0.4262,  0.0721,  1.2026],
        [ 0.0722,  0.4783, -0.6961],
        [ 0.0433,  0.3774,  0.7016],
        [-0.2723, -0.0789,  0.8070],
        [-0.3198,  0.4332, -0.1916],
        [ 0.3172, -0.0526,  1.1023],
        [-0.4494,  0.0200,  0.8497]], grad_fn=<AddmmBackward>)

### 第二種

In [10]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer = []
        self.layer.append(nn.Linear(12,10))
        self.layer.append(nn.BatchNorm1d(10))
        for i in range(10):
            for j in [nn.Linear(10,10), nn.BatchNorm1d(10)]:
                self.layer.append(j)
        self.layer.append(nn.Linear(10, 3))

    def forward(self, x):
        for module in self.layer:
            x = module(x)
        return x

In [11]:
model = Model()
input_ = torch.randn(10,12)
output = model(input_)
output

tensor([[-0.1884, -0.5691, -0.3548],
        [-1.0102, -0.2941, -0.2600],
        [ 0.4972, -0.5921, -0.2062],
        [-0.1011,  0.0855, -0.1735],
        [-0.1702,  0.0491,  0.2049],
        [ 1.3320, -0.0654, -0.7890],
        [ 0.2013, -0.3700, -0.0878],
        [ 1.5039,  0.1614, -0.4090],
        [-0.7895, -0.2793, -0.1924],
        [ 1.1476, -1.0765, -0.6707]], grad_fn=<AddmmBackward>)

### 第三種

In [12]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer = torch.nn.Sequential()
        self.layer.add_module("linear1", nn.Linear(12,10))
        self.layer.add_module("batchnorm1", nn.BatchNorm1d(10))
        for i in range(10):
            self.layer.add_module("linear"+str(2), nn.Linear(10,10))
            self.layer.add_module("batchnorm"+str(2), nn.BatchNorm1d(10))
        self.layer.add_module("linear12", nn.Linear(10, 3))

    def forward(self, x):
        for module in self.layer:
            x = module(x)
        return x

In [13]:
model = Model()
input_ = torch.randn(10,12)
output = model(input_)
output

tensor([[ 0.5500,  0.7641,  0.1462],
        [-0.6844, -0.7846, -0.7649],
        [ 0.6261, -1.0965, -1.1499],
        [-0.6318, -1.5965, -1.0809],
        [-1.0851,  0.2916, -0.2972],
        [-0.4784, -0.5029,  0.3603],
        [-1.0586,  0.1883,  0.3089],
        [ 0.1472,  0.8861,  0.1729],
        [ 0.1502, -0.6747, -0.8878],
        [-0.1534,  0.3835,  0.5255]], grad_fn=<AddmmBackward>)